In [17]:
# https://mne.tools/stable/auto_tutorials/forward/25_automated_coreg.html


import numpy as np

import mne
from mne.coreg import Coregistration
from mne.io import read_info

data_path = mne.datasets.sample.data_path()
# data_path and all paths built from it are pathlib.Path objects
subjects_dir = data_path / "subjects"
subject = "fsaverage"

fname_raw = data_path / "MEG" / 'sample' / "sample_audvis_raw.fif"
fname_raw = 'C:/Users/zhouz\Downloads/CC110033/rest/rest_raw.fif'
info = read_info(fname_raw)
fiducials = mne.io.read_fiducials(fname_raw)
plot_kwargs = dict(
    subject=subject,
    subjects_dir=subjects_dir,
    surfaces="head",
    dig=True,
    eeg=[],
    meg="sensors",
    show_axes=True,
    coord_frame="meg",
)
view_kwargs = dict(azimuth=45, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))

#     surfaces="head-dense"


    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle


        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle


In [24]:
type(fiducials[0])

list

In [14]:
info['dig']

[<DigPoint |        LPA : (-56.7, 0.0, 0.0) mm      : head frame>,
 <DigPoint |     Nasion : (-0.0, 96.3, 0.0) mm      : head frame>,
 <DigPoint |        RPA : (72.7, 0.0, 0.0) mm       : head frame>,
 <DigPoint |     HPI #1 : (52.6, 83.9, 53.2) mm     : head frame>,
 <DigPoint |     HPI #2 : (77.6, -20.6, 11.0) mm    : head frame>,
 <DigPoint |     HPI #3 : (23.0, -81.4, 90.2) mm    : head frame>,
 <DigPoint |     HPI #4 : (-62.0, -20.7, 13.5) mm   : head frame>,
 <DigPoint |     HPI #5 : (-22.8, 96.5, 50.3) mm    : head frame>,
 <DigPoint |   Extra #1 : (-60.4, 30.8, 20.2) mm    : head frame>,
 <DigPoint |   Extra #2 : (-50.9, 62.9, 19.2) mm    : head frame>,
 <DigPoint |   Extra #3 : (-20.4, 99.7, 17.4) mm    : head frame>,
 <DigPoint |   Extra #4 : (7.8, 105.1, 16.1) mm     : head frame>,
 <DigPoint |   Extra #5 : (41.0, 92.4, 18.2) mm     : head frame>,
 <DigPoint |   Extra #6 : (58.8, 67.4, 12.4) mm     : head frame>,
 <DigPoint |   Extra #7 : (70.1, 40.2, 39.4) mm     : head fra

In [25]:
# fiducials = "estimated"  # get fiducials from fsaverage
coreg = Coregistration(info, subject, subjects_dir, fiducials=fiducials[0])
fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

Using high resolution head model in C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using outer_skin.surf for head surface.
Channel types::	grad: 204, mag: 102


In [26]:
coreg.fit_fiducials(verbose=True)
fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

Aligning using fiducials
Start median distance:  18.81 mm
End   median distance:  18.81 mm
Using outer_skin.surf for head surface.
Channel types::	grad: 204, mag: 102


In [27]:
coreg.fit_icp(n_iterations=6, nasion_weight=2.0, verbose=True)
fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

Aligning using ICP
Start     median distance:  18.81 mm
  ICP  1  median distance:  12.50 mm
  ICP  2  median distance:  11.11 mm
  ICP  3  median distance:  11.29 mm
  ICP  4  median distance:  10.55 mm
  ICP  5  median distance:  10.05 mm
  ICP  6  median distance:   9.73 mm
End       median distance:   9.73 mm
Using outer_skin.surf for head surface.
Channel types::	grad: 204, mag: 102


In [11]:
coreg.omit_head_shape_points(distance=5.0 / 1000)  # distance is in meters

Coregistration: Excluding 46 head shape points with distance >= 0.005 m.


In [12]:
coreg.fit_icp(n_iterations=20, nasion_weight=10.0, verbose=True)
fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
mne.viz.set_3d_view(fig, **view_kwargs)

dists = coreg.compute_dig_mri_distances() * 1e3  # in mm
print(
    f"Distance between HSP and MRI (mean/min/max):\n{np.mean(dists):.2f} mm "
    f"/ {np.min(dists):.2f} mm / {np.max(dists):.2f} mm"
)

Aligning using ICP
Start     median distance:   3.66 mm
  ICP  1  median distance:   6.00 mm
  ICP  2  median distance:   7.35 mm
  ICP  3  median distance:   6.95 mm
  ICP  4  median distance:   6.65 mm
  ICP  5  median distance:   6.40 mm
  ICP  6  median distance:   6.30 mm
  ICP  7  median distance:   6.30 mm
  ICP  8  median distance:   6.33 mm
  ICP  9  median distance:   6.30 mm
  ICP 10  median distance:   6.28 mm
End       median distance:   6.28 mm
Channel types::	grad: 204, mag: 102
Distance between HSP and MRI (mean/min/max):
7.53 mm / 0.64 mm / 18.86 mm


In [1]:
mne.write_trans('sample_fsaverage_trans.fif', coreg.trans)

NameError: name 'mne' is not defined

In [28]:
mne.gui.coregistration(subject='fsaverage', subjects_dir=subjects_dir)

Using high resolution head model in C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif.
Using high resolution head model in C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-head-dense.fif
    Triangle neighbors and vertex normals...
Using fiducials from: C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif.
Loading MRI fiducials from C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif... Done!
Using fsaverage-head-dense.fif for head surface.
    1 BEM surfaces found
    Reading a surface...
[done]
    1 BEM surfaces read
Loading MRI fiducials from C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-fiducials.fif... Done!
Using fsaverage-head-dense.fif for head surface.
    1 BEM surface